Code for EE4540 Project assignment 

Standard imports

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib as mpl
from matplotlib.animation import PillowWriter
from IPython.display import Image
from helper import min_radius_for_sensors, graph_is_connected, generate_random_geometric_graph
from randomizedgossip import compute_P_matrix, calculate_W_bar
from randomizedgossip import randomized_gossip_average
from pdmm_average_consensus import pdmm_average_consensus


### • Average Consensus

Suppose the sensor network aims to compute the **average** value of the measurement data.

- Implement a **randomised gossip algorithm** as a baseline method.
- Then, implement the **average consensus problem** using the **PDMM (Primal-Dual Method of Multipliers)** algorithm.
- **Report**: Compare the performance in terms of convergence speed and number of transmissions between the two algorithms.


# Randomized Gossip Implementation

### Add randomized data to the coordinate points

In [ ]:
# Parameters
NUM_SENSORS = 50
AREA_WIDTH = 100  # meters
DIMENSION = 2

# Ensure all sensors are connected with the desired radius
radius = min_radius_for_sensors(NUM_SENSORS, DIMENSION, AREA_WIDTH) 

# Create graph
coords, adjacency = generate_random_geometric_graph(NUM_SENSORS, radius, AREA_WIDTH)

# coords has shape (N, 2)
random_column = np.random.uniform(low=10, high=30, size=(coords.shape[0], 1))  # shape (N, 1)

# concatenate along axis=1 (columns)
coords_augmented = np.hstack((coords, random_column))  # shape (N, DIMENSION + 1)

# Create figure
plt.figure(figsize=(7, 7))

# Scatter plot with color based on random data
sc = plt.scatter(x = coords[:,0], y = coords[:,1], c=coords_augmented[:,2].flatten(), cmap='jet', label='Sensors')

# Draw dashed black lines for connected sensors
for i in range(NUM_SENSORS):
    neighbors = np.where(adjacency[i])[0]
    for j in neighbors:
        if i < j:  # Avoid duplicates
            plt.plot([coords[i, 0], coords[j, 0]],
                     [coords[i, 1], coords[j, 1]],
                     'k--', alpha=0.1)

# Add colorbar
cbar = plt.colorbar(sc)
cbar.set_label('Random Data Value')

# Check if the graph is connected
if graph_is_connected(adjacency):
    print("The graph is connected.")
else:
    print("The graph is NOT connected!")

# Final plot settings
plt.xlim(0, AREA_WIDTH)
plt.ylim(0, AREA_WIDTH)
plt.title(f'Sensor Network with Connections (radius = {radius} m)')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Compute the regular P matrix based on the adjacency matrix
P = adjacency.astype(float) / adjacency.sum(axis=1, keepdims=True)  # Normalize rows to sum to 1

w_bar1 = calculate_W_bar(P, adjacency)

k, gossip_history, real_avg, transmissions = randomized_gossip_average(adjacency, coords_augmented[:, 2], P = P, num_iters=50000, threshold=1e-14)

# Run the randomized gossip algorithm with the optimized weights by inputting matrix P

opt_P = compute_P_matrix(adjacency)

w_bar_opt = calculate_W_bar(opt_P, adjacency)
# Run the randomized gossip algorithm with the optimized P matrix
k, optimal_gossip_history, real_avg, optimal_transmissions = randomized_gossip_average(adjacency, coords_augmented[:, 2], P = opt_P, num_iters=50000, threshold = 1e-14)


# Check if the optimized P performs better than the regular P
if (np.linalg.eigvalsh(w_bar1)[::-1][1] > np.linalg.eigvalsh(w_bar_opt)[::-1][1]):
    print("Optimized P matrix has better convergence properties than the regular P matrix.")
else: 
    print("Regular P matrix has better convergence properties than the optimized P matrix.")
    
# plot the above plots in a single figure with two subplots next to each other
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
axs[0].set_title('Randomized Gossip: Node Values Converging to Average')
for i in range(NUM_SENSORS):
    axs[0].plot([x[i] for x in optimal_gossip_history], alpha=0.5)
axs[0].axhline(real_avg, color='black', linestyle='--', label='True Average', linewidth=1)
axs[0].legend()
axs[0].set_xlabel('Iteration')
axs[0].set_xlim(0, 1000)
axs[0].set_ylabel('Node Value')
axs[0].grid(True)
axs[1].set_title('Randomized Gossip: Node Values vs Transmissions')
for i in range(NUM_SENSORS):
    axs[1].plot(optimal_transmissions, [x[i] for x in optimal_gossip_history], alpha=0.5)
axs[1].axhline(real_avg, color='black', linestyle='--', label='True Average', linewidth=1)
axs[1].legend()
axs[1].set_xlabel('Total Transmissions')
axs[1].set_xlim(0, 2000)
axs[1].set_ylabel('Node Value')
axs[1].grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Calculate the normalized error at each iteration
optimal_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in optimal_gossip_history]

errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in gossip_history]

# Plot on a logarithmic y-axis
plt.figure()
plt.semilogy(optimal_transmissions, optimal_errors, label='Optimized randomized Gossip')
plt.semilogy(transmissions, errors, label='Randomized Gossip (Regular P)')
plt.axhline(1e-14, color='red', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Transmissions')
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)')
plt.title('Convergence of Randomized Gossip Algorithm')
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/convergence_of_randomized_gossip_algorithm.pdf', dpi=600)
plt.show()

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

# Custom frame selection: dense at start, sparser after convergence to reduce video size
frames = []
frames += list(range(0, 50, 1))        # Every 1 frames from 0 to 50
frames += list(range(50, 350, 3))      # Every 10th frame from 10 to 499
frames += list(range(350, 1000, 10))  # Every 300th frame after 500


# Prepare the figure
fig, ax = plt.subplots(figsize=(7, 7))
scat = ax.scatter(coords[:, 0], coords[:, 1], c=optimal_gossip_history[0], cmap='jet', vmin=np.min(optimal_gossip_history[0]), vmax=np.max(optimal_gossip_history[0]))
ax.set_xlim(0, AREA_WIDTH)
ax.set_ylim(0, AREA_WIDTH)
ax.set_title('Randomized Gossip: Node Values Animation')
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.grid(True)

# Draw the static network connections (lines)
for i in range(NUM_SENSORS):
    neighbors = np.where(adjacency[i])[0]
    for j in neighbors:
        if i < j:
            ax.plot([coords[i, 0], coords[j, 0]],
                    [coords[i, 1], coords[j, 1]],
                    'k--', alpha=0.1)

# Add colorbar
cbar = plt.colorbar(scat, ax=ax)
cbar.set_label('Node Value')

# Animation update function
def update(frame):
    values = optimal_gossip_history[frame]
    scat.set_array(values)
    ax.set_title(f'Randomized Gossip: Iteration {frame}')
    return scat

mpl.rcParams['animation.embed_limit'] = 100.0  # default is 20.0 MB

# Create the animation (adjust interval and frames as needed)
ani = animation.FuncAnimation(
    fig, update, frames=frames, interval=100, blit=False, repeat=True
)

plt.close(fig)  # Prevents duplicate static plot

# HTML(ani.to_jshtml())  # Display the animation

# Store the animation with pillow writer
writer = PillowWriter(fps=10)
ani.save('animations/randomized_gossip.gif', writer=writer)  # Save the PDMM animation with broadcasting

# Show the PDMM animation with broadcasting from the written file
Image(filename='animations/randomized_gossip.gif')

In [ ]:
# Run PDMM with broadcasting
k, real_avg, pdmm_history, tx = pdmm_average_consensus(adjacency, coords_augmented[:, 2], num_iters=1000, c = 1, verbose=False, Broadcast=True)

# Plot convergence
plt.figure(figsize=(8, 4))
for i in range(NUM_SENSORS):
    plt.plot([x[i] for x in pdmm_history], alpha=0.5)
plt.axhline(real_avg, color='black', linestyle='--', label='True Average', linewidth=1)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Node Value')
plt.title('PDMM: Node Values Converging to Average')
plt.grid(True)
plt.xlim(0, 100)
plt.show()


# C value optimization

In [ ]:
c = np.linspace(0.01, 1, 100)
# Calculate amount of iterations for each c
iterations = []
transmissions = []
for c_value in tqdm(c):
    k, _, _, tx = pdmm_average_consensus(adjacency, coords_augmented[:, 2], num_iters=1000, c=c_value, verbose=False)
    iterations.append(k)
    transmissions.append(tx[-1])  # Store the last transmission count for each c value



In [ ]:
# print the c value that gives the least number of transmissions
min_transmissions = min(transmissions)

#Calculate optimal c value
optimal_c = c[np.argmin(transmissions)]

print(f'Minimum transmissions: {min_transmissions} for c = {optimal_c}')

#Now put the plots together in a subfigure
fig, axs = plt.subplots(1, 2, figsize=(16, 5))
axs[0].plot(c, iterations)
axs[0].set_xlabel('Step Size (c)')
axs[0].set_ylabel('Number of Iterations to Convergence')
axs[0].set_title('PDMM: Iterations vs Step Size')
axs[0].grid(True)
axs[1].plot(c, transmissions)
axs[1].set_xlabel('Step Size (c)')
axs[1].set_ylabel('Number of Transmissions')
axs[1].set_title('PDMM: Transmissions vs Step Size')
axs[1].grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Create plot with only PDMM: Transmissions vs Step Size
plt.figure()
plt.plot(c, transmissions)
plt.xlabel('Step Size (c)',fontsize=14)
plt.ylabel('Number of Transmissions',fontsize=14)
plt.title('PDMM: Transmissions vs Step Size',fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.savefig('figures/average_consensus_transmissions_vs_step_size.pdf', dpi=600)


In [ ]:
#Use the optimal c value to run PDMM again with broadcasting
k, real_avg, pdmm_broadcast_history, tx = pdmm_average_consensus(adjacency, coords_augmented[:, 2], num_iters=1000, c=optimal_c, verbose=False, Broadcast=True)
# Plot convergence with optimal c
plt.figure(figsize=(8, 4))
for i in range(NUM_SENSORS):
    plt.plot([x[i] for x in pdmm_history], alpha=0.5)
plt.axhline(real_avg, color='black', linestyle='--', label='True Average', linewidth=1)
plt.legend()   
plt.xlabel('Iteration')
plt.ylabel('Node Value')
plt.title(f'PDMM with Optimal c={optimal_c:.2f}: Node Values Converging to Average')
plt.grid(True)
plt.xlim(0, 100)
plt.show()


In [ ]:
# Reset custom frames for PDMM animation with no skipping
frames = list(range(len(pdmm_broadcast_history)))

# Create a new figure for the final PDMM animation
fig, ax = plt.subplots(figsize=(7, 7))
scat = ax.scatter(coords[:, 0], coords[:, 1], c=pdmm_broadcast_history[0], cmap='jet', vmin=np.min(pdmm_broadcast_history[0]), vmax=np.max(pdmm_broadcast_history[0]))
ax.set_xlim(0, AREA_WIDTH)
ax.set_ylim(0, AREA_WIDTH)
ax.set_title('PDMM with broadcasting: Node Values Animation')
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.grid(True)
# Draw the static network connections (lines)
for i in range(NUM_SENSORS):
    neighbors = np.where(adjacency[i])[0]
    for j in neighbors:
        if i < j:
            ax.plot([coords[i, 0], coords[j, 0]],
                    [coords[i, 1], coords[j, 1]],
                    'k--', alpha=0.1)
# Add colorbar
cbar = plt.colorbar(scat, ax=ax)
cbar.set_label('Node Value')
# Animation update function for PDMM
def pdmm_broadcast_update(frame):
    values = pdmm_broadcast_history[frame]
    scat.set_array(values)
    ax.set_title(f'PDMM with broadcasting: Iteration {frame}')
    return scat
# Create the PDMM animation
ani_pdmm = animation.FuncAnimation(
    fig, pdmm_broadcast_update, frames=frames, interval=500, blit=False, repeat=True
)
plt.close(fig)  # Prevents duplicate static plot

# Store the animation with pillow writer
writer = PillowWriter(fps=2)
ani_pdmm.save('animations/pdmm_broadcast_animation.gif', writer=writer)  # Save the PDMM animation with broadcasting

# Show the PDMM animation with broadcasting from the written file
Image(filename='animations/pdmm_broadcast_animation.gif')

In [ ]:
# Use the optimal c value to run PDMM without broadcasting
k, real_avg, pdmm_no_broadcast_history, tx = pdmm_average_consensus(adjacency, coords_augmented[:, 2], num_iters=1000, c=optimal_c, verbose=False, Broadcast=False)
# Plot convergence without broadcasting
plt.figure(figsize=(8, 4))
for i in range(NUM_SENSORS):
    plt.plot([x[i] for x in pdmm_no_broadcast_history], alpha=0.5)
plt.axhline(real_avg, color='black', linestyle='--', label='True Average', linewidth=1)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Node Value')
plt.title(f'PDMM without Broadcasting with Optimal c={optimal_c:.2f}: Node Values Converging to Average')
plt.grid(True)
plt.xlim(0, 100)
plt.show()

In [ ]:
# Reset custom frames for PDMM animation without broadcasting
frames = list(range(len(pdmm_no_broadcast_history)))

# Create a new figure for the PDMM animation without broadcasting
fig, ax = plt.subplots(figsize=(7, 7))
scat = ax.scatter(coords[:, 0], coords[:, 1], c=pdmm_no_broadcast_history[0], cmap='jet', vmin=np.min(pdmm_no_broadcast_history[0]), vmax=np.max(pdmm_no_broadcast_history[0]))
ax.set_xlim(0, AREA_WIDTH)
ax.set_ylim(0, AREA_WIDTH)
ax.set_title('PDMM without Broadcasting: Node Values Animation')
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.grid(True)
# Draw the static network connections (lines)
for i in range(NUM_SENSORS):
    neighbors = np.where(adjacency[i])[0]
    for j in neighbors:
        if i < j:
            ax.plot([coords[i, 0], coords[j, 0]],
                    [coords[i, 1], coords[j, 1]],
                    'k--', alpha=0.1)
# Add colorbar
cbar = plt.colorbar(scat, ax=ax)
cbar.set_label('Node Value')
# Animation update function for PDMM without broadcasting
def pdmm_no_broadcast_update(frame):
    values = pdmm_no_broadcast_history[frame]
    scat.set_array(values)
    ax.set_title(f'PDMM without Broadcasting: Iteration {frame}')
    return scat
# Create the PDMM animation without broadcasting
ani_pdmm_no_broadcast = animation.FuncAnimation(
    fig, pdmm_no_broadcast_update, frames=frames, interval=500, blit=False, repeat=True
)
plt.close(fig)  # Prevents duplicate static plot

# Store the animation with pillow writer
writer = PillowWriter(fps=10)
ani_pdmm_no_broadcast.save('animations/pdmm_no_broadcast_animation.gif', writer=writer)  # Save the PDMM animation without broadcasting

# Show the PDMM animation without broadcasting from the written file
Image(filename='animations/pdmm_no_broadcast_animation.gif')

In [ ]:
#Run randomized gossip and PDMM with broadcasting and without broadcasting, and plot the convergence vs communication cost
# Compute the regular P matrix based on the adjacency matrix
P = adjacency.astype(float) / adjacency.sum(axis=1, keepdims=True)  # Normalize rows to sum to 1

k, gossip_history, real_avg, transmissions = randomized_gossip_average(adjacency, coords_augmented[:, 2], P = P, num_iters=20000, threshold=1e-14)

# Run the randomized gossip algorithm with the optimized weights by inputting matrix P

opt_P = compute_P_matrix(adjacency)
# Run the randomized gossip algorithm with the optimized P matrix
k, optimal_gossip_history, real_avg, optimal_transmissions = randomized_gossip_average(adjacency, coords_augmented[:, 2], P = opt_P, num_iters=20000, threshold=1e-14)

#Calculate the normalized error at each iteration
optimal_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in optimal_gossip_history]

errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in gossip_history]

# Use the optimal c value to run PDMM without broadcasting
k, real_avg, pdmm_no_broadcast_history, pdmm_transmissions_unicast = pdmm_average_consensus(adjacency, coords_augmented[:, 2], num_iters=1000, c=optimal_c, verbose=False, Broadcast=False, threshold=1e-14)

# Use the optimal c value to run PDMM with broadcasting
k, real_avg, pdmm_broadcast_history, pdmm_transmissions_broadcast = pdmm_average_consensus(adjacency, coords_augmented[:, 2], num_iters=1000, c=optimal_c, verbose=False, Broadcast=True, threshold=1e-14)

# Calculate the normalized error at each iteration for PDMM with broadcasting
pdmm_broadcast_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_broadcast_history]
pdmm_no_broadcast_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_no_broadcast_history]


# Plot the error vs transmissions on a logarithmic y-axis
plt.figure()
plt.semilogy(optimal_transmissions, optimal_errors, label='Optimized Randomized Gossip', color='blue')
plt.semilogy(transmissions, errors, label='Randomized Gossip (Regular P)', color='cornflowerblue')
plt.semilogy(pdmm_transmissions_unicast, pdmm_no_broadcast_errors, label='PDMM - Unicast', color = 'salmon')
plt.semilogy(pdmm_transmissions_broadcast, pdmm_broadcast_errors, label='PDMM - Broadcast', color = 'red')
plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title('Average Consensus: Convergence vs Communication Cost',fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_convergence_vs_communication_cost.pdf', bbox_inches='tight', dpi=600)
plt.show()


In [ ]:
# Run randomized gossip with different transmission losses for the optimal P matrix
transmission_losses = [0.0, 0.25, 0.5, 0.75]
plt.figure()

for loss in transmission_losses:
    k, gossip_history_losses, real_avg, transmissions_losses = randomized_gossip_average(
        adjacency, coords_augmented[:, 2], P=opt_P, num_iters=20000, transmissions_loss=loss, threshold=1e-14
    )
    
    # Calculate the normalized error at each iteration for the current transmission loss
    losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in gossip_history_losses]
    
    # Plot the error vs transmissions for the current transmission loss
    
    plt.semilogy(transmissions_losses, losses_errors, label=f'Transmission Loss: {loss}')

plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title(f'Average Consensus with Randomized Gossip\nDifferent Transmission Losses (Optimal P Matrix)', fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_with_randomized_gossip_different_transmission_losses_optimal_p_matrix.pdf', bbox_inches='tight', dpi=600)
plt.show()



In [ ]:
# Run randomized gossip with different transmission losses for the regular P matrix
transmission_losses = [0.0, 0.25, 0.5, 0.75]
plt.figure()

for loss in transmission_losses:
    k, gossip_history_losses, real_avg, transmissions_losses = randomized_gossip_average(
        adjacency, coords_augmented[:, 2], P=P, num_iters=20000, transmissions_loss=loss, threshold=1e-14
    )
    
    # Calculate the normalized error at each iteration for the current transmission loss
    losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in gossip_history_losses]
    
    # Plot the error vs transmissions for the current transmission loss
    
    plt.semilogy(transmissions_losses, losses_errors, label=f'Transmission Loss: {loss}')

plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title(f'Average Consensus with Randomized Gossip\nDifferent Transmission Losses (Regular P Matrix)', fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_with_randomized_gossip_different_transmission_losses_regular_p_matrix.pdf', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
# Run randomized gossip with different transmission losses for the regular P matrix
transmission_losses = [0.0, 0.25, 0.5, 0.75]
plt.figure()

for loss in transmission_losses:
    k, real_avg, pdmm_history_losses, transmissions_losses = pdmm_average_consensus(
        adjacency, coords_augmented[:, 2], num_iters=20000, c=optimal_c, Broadcast = True, transmission_loss=loss, threshold=1e-14, 
    )
    
    # Calculate the normalized error at each iteration for the current transmission loss
    losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_history_losses]
    
    # Plot the error vs transmissions for the current transmission loss
    
    plt.semilogy(transmissions_losses, losses_errors, label=f'Transmission Loss: {loss}')

plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title(f'Average Consensus with Broadcasting PDMM:\n Different Transmission Losses', fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_with_PDMM_broadcast_different_transmission_losses.pdf', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
# Run randomized gossip with different transmission losses for the regular P matrix
transmission_losses = [0.0, 0.25, 0.5, 0.75]
plt.figure()

for loss in transmission_losses:
    k, real_avg, pdmm_history_losses, transmissions_losses = pdmm_average_consensus(
        adjacency, coords_augmented[:, 2], num_iters=20000, c=optimal_c, Broadcast = False, transmission_loss=loss, threshold=1e-14, 
    )
    
    # Calculate the normalized error at each iteration for the current transmission loss
    losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_history_losses]
    
    # Plot the error vs transmissions for the current transmission loss
    
    plt.semilogy(transmissions_losses, losses_errors, label=f'Transmission Loss: {loss}')

plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title(f'Average Consensus with Unicasting PDMM:\n Different Transmission Losses', fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_with_PDMM_unicast_different_transmission_losses.pdf', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
# Run randomized gossip with different transmission losses for the regular P matrix
transmission_losses = [0.0, 0.25, 0.5, 0.75]
plt.figure()

k, real_avg, pdmm_history_losses, transmissions_losses = pdmm_average_consensus(
        adjacency, coords_augmented[:, 2], num_iters=20000, c=optimal_c, Broadcast = True, transmission_loss=0, synchronous = True, threshold=1e-14,  
    )
# Calculate the normalized error at each iteration for the current transmission loss
losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_history_losses]

plt.semilogy(transmissions_losses, losses_errors, label=f'PDMM (synchronous), loss: 0.0')

for loss in transmission_losses:
    k, real_avg, pdmm_history_losses, transmissions_losses = pdmm_average_consensus(
        adjacency, coords_augmented[:, 2], num_iters=20000, c=optimal_c, Broadcast = True, transmission_loss=loss, synchronous = False,threshold=1e-14, 
    )
    
    # Calculate the normalized error at each iteration for the current transmission loss
    losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_history_losses]
    
    # Plot the error vs transmissions for the current transmission loss
    
    plt.semilogy(transmissions_losses, losses_errors, label=f'PDMM (asynchronous), loss: {loss}')

plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title(f'Average Consensus with Asynchronous Broadcasting PDMM:\n Different Transmission Losses', fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_with_PDMM_asynchronous_broadcast_different_transmission_losses.pdf', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
# Run randomized gossip with different transmission losses for the regular P matrix
transmission_losses = [0.0, 0.25, 0.5, 0.75]
plt.figure()

k, real_avg, pdmm_history_losses, transmissions_losses = pdmm_average_consensus(
        adjacency, coords_augmented[:, 2], num_iters=20000, c=optimal_c, Broadcast = False, transmission_loss=0, synchronous = True, threshold=1e-14,  
    )
# Calculate the normalized error at each iteration for the current transmission loss
losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_history_losses]

plt.semilogy(transmissions_losses, losses_errors, label=f'PDMM (synchronous), loss: 0.0')

for loss in transmission_losses:
    k, real_avg, pdmm_history_losses, transmissions_losses = pdmm_average_consensus(
        adjacency, coords_augmented[:, 2], num_iters=20000, c=optimal_c, Broadcast = False, transmission_loss=loss, synchronous = False,threshold=1e-14, 
    )
    
    # Calculate the normalized error at each iteration for the current transmission loss
    losses_errors = [np.linalg.norm(x - real_avg) / np.linalg.norm(real_avg) for x in pdmm_history_losses]
    
    # Plot the error vs transmissions for the current transmission loss
    
    plt.semilogy(transmissions_losses, losses_errors, label=f'PDMM (asynchronous), loss: {loss}')

plt.axhline(1e-14, color='black', linestyle='--', label='Convergence Threshold (1e-14)', linewidth=1)
plt.xlabel('Number of Transmissions', fontsize=14)
plt.ylabel('Normalized Error (||x - avg|| / ||avg||)', fontsize=14)
plt.title(f'Average Consensus with Asynchronous Unicasting PDMM:\n Different Transmission Losses', fontsize=14)
plt.grid(True, which='both', ls='--')
plt.legend()
plt.savefig('figures/average_consensus_with_PDMM_asynchronous_unicast_different_transmission_losses.pdf', bbox_inches='tight', dpi=600)
plt.show()